# Unit 3: Gurdy Robot

<p style="background:#E80000;color:white;">Warning</p>

Before you continue, please kill everything that you have launched and remove Mira Robot from the scene by the methods explained.<br>

<p style="background:#E80000;color:white;">END Warning</p>

To practice and better learn how URDF are created and used, you are going to create, from scratch and with a little help, the Robot Model of Gurdy Robot.

<table style="width:100%">
  <tr>
    <th>
    <figure>
      <img id="fig-A.1" src="img/gurdy_RDS.png" width="300"/>
    </figure>

    </th>
    <th>
        <figure>
      <img id="fig-A.2" src="img/gurdy_RDS2.png" width="300"/>
    </figure>
    </th>

  </tr>
</table>

<p style="background:#EE9023;color:white;">Exercise U3-1</p>

Follow the same procedure that we followed to create the Mira URDF. These are some points to guide you through the creation process:<br>
<ul>
<li>Discover how many links and joints will be needed to create this robot model. Decide which joints will be fixed, revolve, or be continuous.<br>
This Link Tree will give you a good idea of the morphology of Gurdy's link structure.
</li>
</ul>

<img src="img/gurdy_frames.png"/>

And you have an even better view with the PDF generated with the urdf_to_graphiz tool:

[gurdy_link_and_joints](img/gurdy_link_and_joints.pdf)

<ul>
<li>Start by creating a basic geometric URDF model, with no dae files. Use the tools for URDF development.
</li>
</ul>

Here you have a blueprint of the distances to place the most difficult part, which is the joints between the head and the legs. The units are millimeters:

<img src="img/Medidas_Gurdy_head.PNG"/>

You will also need the dimensions of the the basic geometry to have some idea of the way that everything is connected, and the collision elements geometry:<br>
base_link : box size="0.01 0.01 0.01"<br>
head_link : cylinder radius="0.05" length="0.04"<br>
upper_leg_links : cylinder length="0.06" radius="0.0025"<br>
lower_leg_links : cylinder length="0.06" radius="0.0015"<br>
foot_links : sphere radius="0.008"<br>

<ul>
<li>When you have the simple main structure, add the dae models. For this, you will have to copy them from:</li>
</ul>

<ul>
<li>
Once you have the Visual URDF model, it's time to add the collisions, the actuators, and the sensors.<br>
The collisions will be geometric figures, as in the Mira Example.<br>
The actuators are now six instead of three, so make the appropriate changes.<br>
The limits in effort and speed should be around: effort="1.0" and velocity="0.005," but it will depend on the joint.<br>
The weights of the links are:<br>
base_link : mass=None, there is no inertia needed because its a non-functional element.<br>
head_link : mass value="0.01"<br>
upper_leg_links : mass value="0.01"<br>
lower_leg_links : mass value="0.01"<br>
foot_links : mass value="0.01"<br>
</li>
</ul>

<ul>
<li>
Friction is vital in the foot links. Try different values and see the effect ont locomotion. These are some orientative values:<br>
kp = 1000.0<br>
kd = 1000.0<br>
mu1 = 10.0<br>
mu2 = 10.0<br>

</li>
</ul>

<ul>
<li>
PID are tricky. Use the rqt_reconfigure to get the correct values. These are some orientative values:<br>
{p: 3.0, i: 1.0, d: 0.0}
</li>
</ul>

<ul>
<li>
Create a python script that moves the joints as you wish to test that it moves correctly, in an easy and comfortable way.
</li>
</ul>

<ul>
<li>As for sensors, you will have to add an IMU to the "base_link." This is the code for adding an IMU:</li>
</ul>

<p style="background:#EE9023;color:white;">END Exercise U3-1</p>

<p style="background:green;color:white;">Solution Exercise U3.1</p>

Please try to do it by yourself, unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

This is the URDF file:

These are the control values for the weights and efforts stated. If you have a different weight distribution and limit efforts/velocity in the joints, things could change. Remember to use the "rqt_reconfigure" to tune the PID values:

And the control launch would be something like this:

An example of a movement script for Gurdy:

In [ ]:
#!/usr/bin/env python

import rospy
import time
from math import pi, sin, cos, acos
import random
from std_msgs.msg import Float64
from sensor_msgs.msg import JointState
"""
Topics To Write on:
type: std_msgs/Float64
/gurdy/head_upperlegM1_joint_position_controller/command
/gurdy/head_upperlegM2_joint_position_controller/command
/gurdy/head_upperlegM3_joint_position_controller/command
/gurdy/upperlegM1_lowerlegM1_joint_position_controller/command
/gurdy/upperlegM2_lowerlegM2_joint_position_controller/command
/gurdy/upperlegM3_lowerlegM3_joint_position_controller/command
"""

class gurdyJointMover(object):

    def __init__(self):
        rospy.init_node('jointmover_demo', anonymous=True)
        rospy.loginfo("Gurdy JointMover Initialising...")

        self.pub_upperlegM1_joint_position = rospy.Publisher(
            '/gurdy/head_upperlegM1_joint_position_controller/command',
            Float64,
            queue_size=1)
        self.pub_upperlegM2_joint_position = rospy.Publisher(
            '/gurdy/head_upperlegM2_joint_position_controller/command',
            Float64,
            queue_size=1)
        self.pub_upperlegM3_joint_position = rospy.Publisher(
            '/gurdy/head_upperlegM3_joint_position_controller/command',
            Float64,
            queue_size=1)
        self.pub_lowerlegM1_joint_position = rospy.Publisher(
            '/gurdy/upperlegM1_lowerlegM1_joint_position_controller/command',
            Float64,
            queue_size=1)
        self.pub_lowerlegM2_joint_position = rospy.Publisher(
            '/gurdy/upperlegM2_lowerlegM2_joint_position_controller/command',
            Float64,
            queue_size=1)
        self.pub_lowerlegM3_joint_position = rospy.Publisher(
            '/gurdy/upperlegM3_lowerlegM3_joint_position_controller/command',
            Float64,
            queue_size=1)
        joint_states_topic_name = "/gurdy/joint_states"
        rospy.Subscriber(joint_states_topic_name, JointState, self.gurdy_joints_callback)
        gurdy_joints_data = None
        rate = rospy.Rate(2)
        while gurdy_joints_data is None:
            try:
                gurdy_joints_data = rospy.wait_for_message(joint_states_topic_name, JointState, timeout=5)
            except:
                rospy.logwarn("Time out " + str(joint_states_topic_name))
                pass
            rate.sleep()

        self.gurdy_joint_dictionary = dict(zip(gurdy_joints_data.name, gurdy_joints_data.position))

    def move_gurdy_all_joints(self, upperlegM1_angle, upperlegM2_angle, upperlegM3_angle, lowerlegM1_value, lowerlegM2_value ,lowerlegM3_value):
        upperlegM1 = Float64()
        upperlegM1.data = upperlegM1_angle
        upperlegM2 = Float64()
        upperlegM2.data = upperlegM2_angle
        upperlegM3 = Float64()
        upperlegM3.data = upperlegM3_angle

        lowerlegM1 = Float64()
        lowerlegM1.data = lowerlegM1_value
        lowerlegM2 = Float64()
        lowerlegM2.data = lowerlegM2_value
        lowerlegM3 = Float64()
        lowerlegM3.data = lowerlegM3_value

        self.pub_upperlegM1_joint_position.publish(upperlegM1)
        self.pub_upperlegM2_joint_position.publish(upperlegM2)
        self.pub_upperlegM3_joint_position.publish(upperlegM3)

        self.pub_lowerlegM1_joint_position.publish(lowerlegM1)
        self.pub_lowerlegM2_joint_position.publish(lowerlegM2)
        self.pub_lowerlegM3_joint_position.publish(lowerlegM3)


    def gurdy_joints_callback(self, msg):
        """
        sensor_msgs/JointState
        std_msgs/Header header
        uint32 seq
        time stamp
        string frame_id
        string[] name
        float64[] position
        float64[] velocity
        float64[] effort

        :param msg:
        :return:
        """
        self.gurdy_joint_dictionary = dict(zip(msg.name, msg.position))


    def convert_angle_to_unitary(self, angle):
        """
        Removes complete revolutions from angle and converts to positive equivalent
        if the angle is negative
        :param angle: Has to be in radians
        :return:
        """
        # Convert to angle between [0,360)
        complete_rev = 2 * pi
        mod_angle = int(angle / complete_rev)
        clean_angle = angle - mod_angle * complete_rev
        # Convert Negative angles to their corresponding positive values
        if clean_angle < 0:
            clean_angle += 2 * pi

        return clean_angle

    def assertAlmostEqualAngles(self, x, y,):
        c2 = (sin(x) - sin(y)) ** 2 + (cos(x) - cos(y)) ** 2
        angle_diff = acos((2.0 - c2) / 2.0)
        return angle_diff

    def gurdy_check_continuous_joint_value(self, joint_name, value, error=0.1):
        """
        Check the joint by name 'base_waist_joint', 'body_head_joint', 'waist_body_joint is near the value given
        We have to convert the joint values removing whole revolutions and converting negative versions
        of the same angle
        :param joint_name:
        :param value:
        :param error: In radians
        :return:
        """
        joint_reading = self.gurdy_joint_dictionary.get(joint_name)
        if not joint_reading:
            print "self.gurdy_joint_dictionary="+str(self.gurdy_joint_dictionary)
            print "joint_name===>"+str(joint_name)
            assert "There is no data about that joint"
        clean_joint_reading = self.convert_angle_to_unitary(angle=joint_reading)
        clean_value = self.convert_angle_to_unitary(angle=value)

        dif_angles = self.assertAlmostEqualAngles(clean_joint_reading, clean_value)
        similar = dif_angles <= error

        return similar

    def gurdy_movement_look(self, upperlegM1_angle, upperlegM2_angle, upperlegM3_angle, lowerlegM1_value, lowerlegM2_value ,lowerlegM3_value):
        """
        Move:
        'head_upperlegM1_joint',
        'head_upperlegM2_joint',
        'head_upperlegM3_joint',
        'upperlegM1_lowerlegM1_joint',
        'upperlegM2_lowerlegM2_joint',
        'upperlegM3_lowerlegM3_joint'
        :return:
        """
        check_rate = 5.0
        position_upperlegM1 = upperlegM1_angle
        position_upperlegM2 = upperlegM2_angle
        position_upperlegM3 = upperlegM3_angle

        position_lowerlegM1 = lowerlegM1_value
        position_lowerlegM2 = lowerlegM2_value
        position_lowerlegM3 = lowerlegM3_value

        similar_upperlegM1 = False
        similar_upperlegM2 = False
        similar_upperlegM3 = False

        similar_lowerlegM1 = False
        similar_lowerlegM2 = False
        similar_lowerlegM3 = False

        rate = rospy.Rate(check_rate)
        while not (similar_upperlegM1 and similar_upperlegM2 and similar_upperlegM3 and similar_lowerlegM1 and similar_lowerlegM2 and similar_lowerlegM3):
            self.move_gurdy_all_joints(position_upperlegM1,
                                       position_upperlegM2,
                                       position_upperlegM3,
                                       position_lowerlegM1,
                                       position_lowerlegM2,
                                       position_lowerlegM3)
            similar_upperlegM1 = self.gurdy_check_continuous_joint_value(joint_name="head_upperlegM1_joint",
                                                                         value=position_upperlegM1)
            similar_upperlegM2 = self.gurdy_check_continuous_joint_value(joint_name="head_upperlegM2_joint",
                                                                         value=position_upperlegM2)
            similar_upperlegM3 = self.gurdy_check_continuous_joint_value(joint_name="head_upperlegM3_joint",
                                                                         value=position_upperlegM3)
            similar_lowerlegM1 = self.gurdy_check_continuous_joint_value(joint_name="upperlegM1_lowerlegM1_joint",
                                                                         value=position_lowerlegM1)
            similar_lowerlegM2 = self.gurdy_check_continuous_joint_value(joint_name="upperlegM2_lowerlegM2_joint",
                                                                         value=position_lowerlegM2)
            similar_lowerlegM3 = self.gurdy_check_continuous_joint_value(joint_name="upperlegM3_lowerlegM3_joint",
                                                                         value=position_lowerlegM3)

            rate.sleep()

    def gurdy_init_pos_sequence(self):
        """
        UPPER limits lower="-1.55" upper="0.0"
        LOWER limits lower="-2.9" upper="1.5708"
        :return:
        """
        upperlegM1_angle = -1.55
        upperlegM2_angle = -1.55
        upperlegM3_angle = -1.55
        lowerlegM1_angle = 0.0
        lowerlegM2_angle = 0.0
        lowerlegM3_angle = 0.0
        self.gurdy_movement_look(upperlegM1_angle,
                                 upperlegM2_angle,
                                 upperlegM3_angle,
                                 lowerlegM1_angle,
                                 lowerlegM2_angle,
                                 lowerlegM3_angle)

        lowerlegM1_angle = -1.55
        lowerlegM2_angle = -1.55
        lowerlegM3_angle = -1.55
        self.gurdy_movement_look(upperlegM1_angle,
                                 upperlegM2_angle,
                                 upperlegM3_angle,
                                 lowerlegM1_angle,
                                 lowerlegM2_angle,
                                 lowerlegM3_angle)

    def gurdy_hop(self, num_hops=15):
        """
        UPPER limits lower="-1.55" upper="0.0"
        LOWER limits lower="-2.9" upper="1.5708"
        :return:
        """

        upper_delta = 1
        basic_angle = -1.55
        angle_change = random.uniform(0.2, 0.7)
        upperlegM_angle = basic_angle
        lowerlegM_angle = basic_angle - upper_delta * angle_change * 2.0

        #self.gurdy_init_pos_sequence()
        for repetitions in range(num_hops):
            self.gurdy_movement_look(upperlegM_angle,
                                     upperlegM_angle,
                                     upperlegM_angle,
                                     lowerlegM_angle,
                                     lowerlegM_angle,
                                     lowerlegM_angle)

            upper_delta *= -1
            if upper_delta < 0:
                upperlegM_angle = basic_angle + angle_change
            else:
                upperlegM_angle = basic_angle
            lowerlegM_angle = basic_angle - upper_delta * angle_change * 2.0


    def gurdy_moverandomly(self):
        """
        UPPER limits lower="-1.55" upper="0.0"
        LOWER limits lower="-2.9" upper="1.5708"
        :return:
        """
        upperlegM1_angle = random.uniform(-1.55, 0.0)
        upperlegM2_angle = random.uniform(-1.55, 0.0)
        upperlegM3_angle = random.uniform(-1.55, 0.0)
        lowerlegM1_angle = random.uniform(-2.9, pi/2)
        lowerlegM2_angle = random.uniform(-2.9, pi/2)
        lowerlegM3_angle = random.uniform(-2.9, pi/2)
        self.gurdy_movement_look(upperlegM1_angle,
                                 upperlegM2_angle,
                                 upperlegM3_angle,
                                 lowerlegM1_angle,
                                 lowerlegM2_angle,
                                 lowerlegM3_angle)

    def movement_random_loop(self):
        """
        Executed movements in a random way
        :return:
        """
        rospy.loginfo("Start Moving Gurdy...")
        while not rospy.is_shutdown():
            self.gurdy_init_pos_sequence()
            #self.gurdy_moverandomly()
            self.gurdy_hop()

if __name__ == "__main__":
    gurdy_jointmover_object = gurdyJointMover()
    gurdy_jointmover_object.movement_random_loop()



If everything went well, you should have something similar to this:

<img src="img/gurdy_demo.gif"/>

<p style="background:#EE9023;color:white;">Exercise U3-2</p>

Create alternative versions of Gurdy by following these ideas:<br>
<ol>
<li>Add an extra joint in each upper leg that allows it to rotate on the Z axis and change the angle at will.</li>
<li>Create an alternative version of Gurdy with an extra arm on the top of the head, and place a camera on the tip of the arm.</li>
<li>Change the foot joints from fix to primatic so that it can change the length of its legs.</li>
</ol>


<p style="background:#EE9023;color:white;">END Exercise U3-2</p>

<p style="background:#EE9023;color:white;">Exercise U3-3</p>

<ol>
<li>Create an algorithm that moves Gurdy in the direction that you desire through a cmd_vel topic publishing.</li>
<li>Create a function that detects data through the IMU when it has fallen and is upside-down, and starts a recovery routine to get on its feet again.</li>
<li>Try to make him able to jump high. Bear in mind that you will have to change effort limits, PID values, and so on, without losing control of Gurdy.</li>
</ol>

<p style="background:#EE9023;color:white;">END Exercise U3-3</p>